# Latihan

Gunakan data dari House Price (https://www.kaggle.com/c/house-prices-advanced-regression-techniques) yang pernah digunakan sebelumnya, gunakan model perceptron untuk mendapat hasil prediksinya lalu submit hasilnya.

In [1]:
import numpy as np
import pandas as pd

In [103]:
raw_data = pd.read_csv('dataset/train.csv')
test_data = pd.read_csv('dataset/test.csv')

# change data type column MSSubClass into object 
raw_data['MSSubClass'] = raw_data['MSSubClass'].astype('object')
test_data['MSSubClass'] = test_data['MSSubClass'].astype('object')

# remove and save column Id
y = raw_data['SalePrice'].copy() # untuk Label
raw_data.drop(columns=['Id', 'SalePrice'], inplace=True)
test_Id = test_data['Id'].copy() # untuk submit kaggle
test_data.drop(columns='Id', inplace=True)

# combine raw_data (train) and test_data to single dataframe
combine_data = pd.concat([raw_data.copy(), test_data.copy()])
index_raw_data = list(range(len(raw_data)))
index_test_data = list(range(len(raw_data), len(combine_data)))

# FillNa categorical columns with 'NotAvail'
columns_categorical = combine_data.select_dtypes('object').columns.values.tolist()
for column in columns_categorical:
    combine_data[column].fillna('NotAvail', inplace=True)
    
# FillNa numeric columns with 0 except 'LotFrontage' column
columns_numeric = combine_data.select_dtypes('number').columns.values.tolist()
for column in columns_numeric:
    if column == 'LotFrontage':
        combine_data[column].fillna(raw_data['LotFrontage'].median(skipna=True), inplace=True)
    else:
        combine_data[column].fillna(0, inplace=True)

# one hot encoding categorical columns
combine_data = pd.get_dummies(combine_data)

# split combine data
raw_data = combine_data.iloc[index_raw_data, :].copy()
test_data = combine_data.iloc[index_test_data, :].copy()

In [111]:
# normalization
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

normalize_X = raw_data.values
normalize_y = y.values.reshape((1460, 1))
normalize_X = X_scaler.fit_transform(normalize_X)
normalize_y = y_scaler.fit_transform(normalize_y)

normalize_y = normalize_y.reshape(1460)

In [137]:
# Create model 1
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(input_dim=311, units=1, kernel_initializer='uniform', activation='linear'))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [137]:
# Create model 2
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(input_dim=311, units=1, kernel_initializer='uniform', activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
# mae: 0.017

In [159]:
# Create model 3
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(input_dim=311, units=1, kernel_initializer='uniform', activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
# mae: 0.168

In [191]:
# Create model 4
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(input_dim=311, units=1, kernel_initializer='uniform', activation='relu'))
model.add(Dense(400, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
# mae: 0.168

In [ ]:
# Train
model.fit(normalize_X, normalize_y, epochs=100)

Epoch 1/100
1460/1460 [==============================] - 0s 206us/step - loss: 7.5940e-04 - mae: 0.0191
Epoch 2/100
1460/1460 [==============================] - 0s 170us/step - loss: 7.8533e-04 - mae: 0.0196
Epoch 3/100
1460/1460 [==============================] - 0s 166us/step - loss: 8.7855e-04 - mae: 0.0207
Epoch 4/100
1460/1460 [==============================] - 0s 150us/step - loss: 9.1208e-04 - mae: 0.0215
Epoch 5/100
1460/1460 [==============================] - 0s 161us/step - loss: 9.4060e-04 - mae: 0.0213
Epoch 6/100
1460/1460 [==============================] - 0s 159us/step - loss: 9.1683e-04 - mae: 0.0209
Epoch 7/100
1460/1460 [==============================] - 0s 162us/step - loss: 9.3902e-04 - mae: 0.0217
Epoch 8/100
1460/1460 [==============================] - 0s 149us/step - loss: 7.7327e-04 - mae: 0.0194
Epoch 9/100
1460/1460 [==============================] - 0s 174us/step - loss: 7.1202e-04 - mae: 0.0184
Epoch 10/100
1460/1460 [==============================] - 0s 177

In [182]:
# normalize test_data
test_values = test_data.values
test_values = X_scaler.transform(test_values)

# predict data test
test_predict = model.predict(test_values)

# inverse transform predict
test_predict = test_predict.reshape((1459, 1))
test_predict = y_scaler.inverse_transform(test_predict)
test_predict = test_predict.reshape(1459)

In [183]:
# Kaggle Submission
kaggle_submission = pd.DataFrame({
    'Id': test_Id,
    'SalePrice': test_predict
})

kaggle_submission.to_csv('neural_network4.csv', index=False)